# Lab 3-1: Data Drift Analysis (Jupyter Notebook)

이 노트북에서는 **Data Drift를 대화형으로 분석**합니다.

## 📋 실습 내용

1. 데이터 로드 및 탐색
2. Reference vs Current 데이터 분할
3. 통계적 비교
4. Drift 시뮬레이션
5. **KS Test를 이용한 Drift 감지**
6. Feature별 Drift 분석
7. 분포 시각화
8. MLflow에 메트릭 기록

---

## 1. 환경 설정

필요한 패키지를 설치하고 import합니다.

In [ ]:
# 패키지 설치
!pip install kfp==1.8.22 mlflow==2.9.2 \
    "pydantic>=1.8.2,<2.0.0" "protobuf<5,>=3.12.0" \
    "numpy>=1.26,<2.0" scipy scikit-learn pandas \
    matplotlib seaborn -q

print("✅ 패키지 설치 완료!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from scipy.stats import ks_2samp
import mlflow
import warnings
warnings.filterwarnings('ignore')

# Plot 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Import 완료")

---

## 2. 데이터 로드 및 탐색

California Housing 데이터셋을 사용합니다.

In [ ]:
# 데이터 로드
data = fetch_california_housing(as_frame=True)
df = data.frame

print(f"📊 Total samples: {len(df):,}")
print(f"📊 Features: {len(df.columns)-1}")
print(f"\n컬럼 목록:")
for col in df.columns:
    print(f"  - {col}")

In [ ]:
# 데이터 샘플 확인
df.head(10)

In [ ]:
# 통계 요약
df.describe()

In [ ]:
# Feature 분포 시각화
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for idx, col in enumerate(df.columns):
    axes[idx].hist(df[col], bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 전체 데이터 분포 확인 완료")

---

## 3. Reference vs Current 데이터 분할

- **Reference data**: 모델 학습 시 사용한 "과거" 데이터
- **Current data**: 프로덕션에서 수집된 "현재" 데이터

In [ ]:
# Reference data (과거 데이터)
reference_data = df.sample(n=5000, random_state=42)

# Current data (현재 데이터)
current_data = df.sample(n=3000, random_state=123)

print(f"✅ Reference data: {len(reference_data):,} samples")
print(f"✅ Current data: {len(current_data):,} samples")

---

## 4. 통계적 비교 (Drift 전)

먼저 Drift를 시뮬레이션하기 전에 두 데이터셋을 비교합니다.

In [ ]:
# MedInc Feature 비교
print("=" * 60)
print("  MedInc (Median Income) 통계 비교")
print("=" * 60)
print("\nReference Data:")
print(reference_data['MedInc'].describe())
print("\nCurrent Data:")
print(current_data['MedInc'].describe())
print("\n📊 평균 차이:", 
      f"{abs(reference_data['MedInc'].mean() - current_data['MedInc'].mean()):.4f}")

In [ ]:
# 분포 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(reference_data['MedInc'], bins=50, alpha=0.6, 
             label='Reference', color='blue', edgecolor='black')
axes[0].hist(current_data['MedInc'], bins=50, alpha=0.6, 
             label='Current', color='red', edgecolor='black')
axes[0].set_xlabel('MedInc', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('MedInc Distribution (Before Drift)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
data_comparison = pd.DataFrame({
    'Reference': reference_data['MedInc'],
    'Current': current_data['MedInc']
})
axes[1].boxplot([data_comparison['Reference'], data_comparison['Current']], 
                labels=['Reference', 'Current'])
axes[1].set_ylabel('MedInc', fontsize=12)
axes[1].set_title('MedInc Box Plot Comparison', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Drift 전 분포는 거의 동일합니다")

---

## 5. Drift 시뮬레이션

현실에서는 시간이 지나면서 데이터 분포가 변합니다.
예시: 새로운 지역의 데이터가 추가되어 소득(MedInc) 분포가 변경

In [ ]:
# Current data에 Drift 주입
current_data_drifted = current_data.copy()

# MedInc를 1.5배 증가 + 노이즈 추가
drift_strength = 1.5
noise_std = 0.3

current_data_drifted['MedInc'] = (
    current_data_drifted['MedInc'] * drift_strength + 
    np.random.normal(0, noise_std, len(current_data_drifted))
)

print("=" * 60)
print("  Drift 시뮬레이션 완료")
print("=" * 60)
print(f"\n변경 사항:")
print(f"  - MedInc × {drift_strength}")
print(f"  - 노이즈 추가: N(0, {noise_std})")
print(f"\n평균 변화:")
print(f"  - Before: {current_data['MedInc'].mean():.4f}")
print(f"  - After: {current_data_drifted['MedInc'].mean():.4f}")
print(f"  - 증가율: {(current_data_drifted['MedInc'].mean() / current_data['MedInc'].mean() - 1) * 100:.1f}%")

In [ ]:
# Drift 후 분포 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(reference_data['MedInc'], bins=50, alpha=0.6, 
             label='Reference', color='blue', edgecolor='black')
axes[0].hist(current_data_drifted['MedInc'], bins=50, alpha=0.6, 
             label='Current (Drifted)', color='red', edgecolor='black')
axes[0].set_xlabel('MedInc', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('MedInc Distribution (After Drift)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
data_comparison_drift = pd.DataFrame({
    'Reference': reference_data['MedInc'],
    'Current (Drifted)': current_data_drifted['MedInc']
})
axes[1].boxplot([data_comparison_drift['Reference'], 
                 data_comparison_drift['Current (Drifted)']], 
                labels=['Reference', 'Current\n(Drifted)'])
axes[1].set_ylabel('MedInc', fontsize=12)
axes[1].set_title('MedInc Box Plot (After Drift)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Drift 후 분포가 명확히 다릅니다!")

---

## 6. Kolmogorov-Smirnov Test로 Drift 감지

KS Test는 두 분포가 같은지 통계적으로 검정합니다.
- **KS Statistic**: 0~1 사이의 값, 클수록 분포 차이가 큼
- **P-value**: < 0.05이면 통계적으로 유의미한 차이 (Drift)

In [ ]:
# Feature별 Drift 계산 (Kolmogorov-Smirnov Test)
drift_results = []

for col in reference_data.columns:
    # KS Test
    statistic, p_value = ks_2samp(
        reference_data[col], 
        current_data_drifted[col]
    )
    
    drift_detected = p_value < 0.05  # p < 0.05이면 Drift
    
    drift_results.append({
        'Feature': col,
        'KS Statistic': statistic,
        'P-value': p_value,
        'Drift Detected': drift_detected
    })

drift_df = pd.DataFrame(drift_results)
drift_df = drift_df.sort_values('KS Statistic', ascending=False)

print("=" * 80)
print("  Feature별 Drift Analysis (KS Test)")
print("=" * 80)
print(drift_df.to_string(index=False))
print("\n📊 Drift 감지된 Feature 수:", drift_df['Drift Detected'].sum())

In [ ]:
# Drift Score 시각화
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['red' if d else 'green' for d in drift_df['Drift Detected']]
bars = ax.barh(drift_df['Feature'], drift_df['KS Statistic'], color=colors, alpha=0.7, edgecolor='black')

ax.axvline(x=0.1, color='red', linestyle='--', linewidth=2, label='Typical Threshold')
ax.set_xlabel('KS Statistic (Drift Score)', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)
ax.set_title('Drift Score by Feature', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("✅ MedInc가 가장 큰 Drift를 보입니다!")

---

## 7. 전체 Feature 분포 비교

모든 Feature의 분포를 한눈에 비교합니다.

In [ ]:
# 모든 Feature 분포 비교
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(reference_data.columns):
    axes[idx].hist(reference_data[col], bins=30, alpha=0.6, 
                   label='Reference', color='blue', edgecolor='black')
    axes[idx].hist(current_data_drifted[col], bins=30, alpha=0.6, 
                   label='Current', color='red', edgecolor='black')
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].legend(fontsize=9)
    axes[idx].grid(True, alpha=0.3)
    
    # Drift 여부 표시
    drift_status = drift_df[drift_df['Feature'] == col]['Drift Detected'].values[0]
    if drift_status:
        axes[idx].set_facecolor('#ffcccc')  # 빨간 배경

plt.suptitle('Distribution Comparison: Reference vs Current (Red background = Drift detected)', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("✅ 전체 Feature 분포 비교 완료")

---

## 8. MLflow에 메트릭 기록

Drift 결과를 MLflow에 기록하여 추적합니다.

In [ ]:
# MLflow 설정
MLFLOW_TRACKING_URI = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000"
EXPERIMENT_NAME = "drift-monitoring"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"✅ MLflow URI: {MLFLOW_TRACKING_URI}")
print(f"✅ Experiment: {EXPERIMENT_NAME}")

In [ ]:
# MLflow Run 생성 및 메트릭 기록
with mlflow.start_run(run_name="jupyter_drift_analysis"):
    # 전체 Drift Score
    n_drifted = drift_df['Drift Detected'].sum()
    drift_score = n_drifted / len(drift_df)
    
    mlflow.log_metric("drift_score", drift_score)
    mlflow.log_metric("n_drifted_features", n_drifted)
    mlflow.log_metric("n_total_features", len(drift_df))
    
    # Feature별 Drift Score
    for _, row in drift_df.iterrows():
        feature = row['Feature']
        mlflow.log_metric(f"ks_statistic_{feature}", row['KS Statistic'])
        mlflow.log_metric(f"p_value_{feature}", row['P-value'])
    
    # 파라미터
    mlflow.log_param("reference_size", len(reference_data))
    mlflow.log_param("current_size", len(current_data_drifted))
    mlflow.log_param("drift_strength", drift_strength)
    mlflow.log_param("noise_std", noise_std)
    
    # 태그
    mlflow.set_tag("drift_detected", str(n_drifted > 0))
    mlflow.set_tag("notebook", "true")
    mlflow.set_tag("method", "ks_test")
    
    print("=" * 60)
    print("  MLflow에 기록 완료")
    print("=" * 60)
    print(f"\n📊 Metrics:")
    print(f"  - drift_score: {drift_score:.2f}")
    print(f"  - n_drifted_features: {n_drifted}")
    print(f"  - n_total_features: {len(drift_df)}")
    print(f"\n✅ MLflow UI에서 확인하세요!")
    print(f"   kubectl port-forward svc/mlflow-server-service -n mlflow-system 5000:5000")
    print(f"   http://localhost:5000")

---

## 9. 실습 과제

### 과제 1: Drift Strength 변경
위의 Section 5에서 `drift_strength`를 다른 값으로 변경하여 실행해보세요.
- `drift_strength = 1.2` (약한 Drift)
- `drift_strength = 2.0` (강한 Drift)

### 과제 2: 다른 Feature에 Drift 주입
`AveRooms` Feature에도 Drift를 추가해보세요.

```python
current_data_drifted['AveRooms'] = current_data_drifted['AveRooms'] * 0.8
```

### 과제 3: Drift Threshold 조정
KS Test에서 p-value threshold를 0.01로 변경하여 더 엄격하게 Drift를 감지해보세요.

---

## 10. 정리

### 학습한 내용
1. ✅ Data Drift의 개념과 중요성
2. ✅ **Kolmogorov-Smirnov Test를 이용한 Drift 감지**
3. ✅ Feature별 Drift 분석
4. ✅ 통계적 유의성 검정 (P-value)
5. ✅ MLflow를 이용한 Drift 추적

### 주요 메트릭
- **Drift Score**: Drift가 감지된 Feature 비율
- **KS Statistic**: 두 분포의 차이 정도 (0~1)
- **P-value**: 통계적 유의성 (< 0.05이면 Drift)

### 다음 단계
- `../scripts/2_monitor_pipeline.py` 실행하여 Kubeflow Pipeline 생성
- `../scripts/3_retrain_pipeline.py` 실행하여 자동 재학습 구현

---

**실습 완료! 🎉**